In [1]:
import numpy as np
import random
import pandas as pd
import unidecode
import getpass
import smtplib
from string import Template
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
def mail_formating(string):
    string = unidecode.unidecode(string)
    string = string.lower()
    return string

In [3]:
def load_mail_template(language):
    file_name = language + '_mail_template.txt'
    with open(file_name, mode='r', encoding='utf-8') as file:
        mail = Template(file.read())
    return mail 

In [4]:
def random_derangement(n):
    while True:
        v = np.arange(n)
        for j in range(n - 1, -1, -1):
            p = random.randint(0, j)
            if v[p] == j:
                break
            else:
                v[j], v[p] = v[p], v[j]
        else:
            if v[0] != 0:
                return v

In [5]:
name_list = pd.read_csv("name_list.csv", header=0)

In [6]:
name_list['Email address'] = name_list.apply(lambda row: mail_formating(row['First name']) + '.' +
                                                       mail_formating(row['Last name']) + '@lip6.fr', axis=1)

In [7]:
random.seed(0)
der = random_derangement(len(name_list))
name_list['To'] = name_list['First name'].iloc[der].reset_index(drop=True)

In [8]:
fr_mail_subject = '[Secret Santa] Tu dois faire un cadeau à...'
en_mail_subject = '[Secret Santa] You must give a present to...'

In [9]:
fr_mail = load_mail_template('fr')
en_mail = load_mail_template('en')

In [22]:
user_email = input('Enter your email address: ')
user_password = getpass.getpass("Enter your e-mail password: ")

Enter your email address: marvin.lasserre@gmail.com
Enter your e-mail password: ········


In [23]:
try:
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.ehlo()
    server.starttls()
    server.login(user_email, user_password)
    print("Successfully connected to server")
except:
    print("Failed to connect to server")    

Successfully connected to server


In [28]:
for index, row in name_list.iterrows():
    #if row['First name'] != 'Marvin':
        #continue
    msg = MIMEMultipart() # create a message
    
    # setup the parameters of the message
    msg['From'] = user_email
    msg['To'] = row['Email address']
    
    # add in the actual person name to the message template
    if row['Language'] == 'en':
        custom_mail = en_mail.substitute(SANTA_NAME=row['First name'], RECIPIENT_NAME=row['To'])
        msg['Subject'] = en_mail_subject
    elif row['Language'] == 'fr':
        custom_mail = fr_mail.substitute(SANTA_NAME=row['First name'], RECIPIENT_NAME=row['To'])
        msg['Subject'] = fr_mail_subject
        
    print(msg)
    
    # add in the message body
    msg.attach(MIMEText(custom_mail, 'plain'))
        
    # send the message via the server set up earlier.
    #server.send_message(msg)
    print("Send mail to ", row['First name'], ' ', row['Last name'])
    del msg

Content-Type: multipart/mixed; boundary="===============2044857532908331599=="
MIME-Version: 1.0
From: marvin.lasserre@gmail.com
To: kostantinos.dogeas@lip6.fr
Subject: [Secret Santa] You must give a present to...

--===============2044857532908331599==

--===============2044857532908331599==--

Send mail to  Kostantinos   Dogeas
Content-Type: multipart/mixed; boundary="===============7409550268663459575=="
MIME-Version: 1.0
From: marvin.lasserre@gmail.com
To: anne-elisabeth.falq@lip6.fr
Subject: =?utf-8?q?=5BSecret_Santa=5D_Tu_dois_faire_un_cadeau_=C3=A0=2E=2E=2E?=

--===============7409550268663459575==

--===============7409550268663459575==--

Send mail to  Anne-Elisabeth   Falq
Content-Type: multipart/mixed; boundary="===============1314278795803408338=="
MIME-Version: 1.0
From: marvin.lasserre@gmail.com
To: gaspard.ducamp@lip6.fr
Subject: =?utf-8?q?=5BSecret_Santa=5D_Tu_dois_faire_un_cadeau_=C3=A0=2E=2E=2E?=

--===============1314278795803408338==

--===============13142787958034

In [27]:
try:
    server.close()
    print("Successfuly disconnected from server")
except:
    print("Failed to disconnect from server")

Successfuly disconnected from server
